# Exploration15
## 내가 좋아하는 영화 추천받기
### 필요한 모듈 불러오기

In [29]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


### 데이터 불러온 뒤 전처리하기

In [30]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [31]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [32]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 데이터 분석하기

In [33]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [34]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [35]:
# movies와 ratings를 합쳐준다.
data= pd.merge(movies, ratings)

In [36]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = [ 'user_id','title', 'ratings']
data = data[using_cols]
data.head(10)

,user_id,title,ratings
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [37]:
data['title'] = data['title'].str.lower() 

In [38]:
#가장 인기있는 영화 30개(인기순)
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [39]:
# 유저별 몇 편의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

### 선호하는 영화 선택하기

In [40]:
a= set(data['title'])

In [41]:
a

{'rollercoaster (1977)',
 'dry cleaning (nettoyage à sec) (1997)',
 'aliens (1986)',
 'night at the roxbury, a (1998)',
 'roustabout (1964)',
 'rent-a-kid (1995)',
 'dr. no (1962)',
 'moonlight and valentino (1995)',
 'santa claus: the movie (1985)',
 'haunted world of edward d. wood jr., the (1995)',
 "mummy's ghost, the (1944)",
 'angels and insects (1995)',
 'jaws (1975)',
 'my blue heaven (1990)',
 'primary colors (1998)',
 'hamlet (1990)',
 'to wong foo, thanks for everything! julie newmar (1995)',
 'long kiss goodnight, the (1996)',
 'kull the conqueror (1997)',
 'alien escape (1995)',
 'prom night iv: deliver us from evil (1992)',
 'playing god (1997)',
 'urban legend (1998)',
 'last klezmer: leopold kozlowski, his life and music, the (1995)',
 'sandpiper, the (1965)',
 'favor, the (1994)',
 'misérables, les (1998)',
 'ghostbusters ii (1989)',
 'super mario bros. (1993)',
 'bachelor party (1984)',
 'to catch a thief (1955)',
 'screwed (2000)',
 'beauty (1998)',
 "jesus' son (199

In [42]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가한다.
my_favorite = ['rollercoaster (1977)' , 'aliens (1986)' ,'santa claus: the movie (1985)' ,
               'hamlet (1990)' ,'kull the conqueror (1997)']

# 'kimin'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['jang']*5, 'title': my_favorite, 'ratings':[5]*5})

# user_id에 'kimin'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
if not data.isin({'user_id':['jang']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(20) 

,user_id,title,ratings
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


### 모델에 활용하기위한 전처리하기.

In [43]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [44]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['jang'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['kull the conqueror (1997)'])

6039
1445


In [45]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,ratings
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,2279,5
1,6039,1084,5
2,6039,2158,5
3,6039,3406,5


### CSR metrix 만들기

In [46]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.ratings, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련하기

In [48]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [61]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [62]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [63]:
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

In [64]:
jang, back_to_the_future = user_to_idx['jang'], title_to_idx['kull the conqueror (1997)']
jang_vector, back_to_the_future_vector = als_model.user_factors[jang], als_model.item_factors[back_to_the_future]

In [65]:
jang_vector

array([ 0.2995162 , -0.11275924, -0.38618067,  0.22861867,  0.17488368,
       -0.22059843,  0.2632128 ,  0.11074856,  0.34350044,  0.06503578,
        0.345084  , -0.12875293,  0.10714889,  0.00281918, -0.33819953,
       -0.39113712, -0.29414758,  0.12951429, -0.22310273,  0.06568135,
        0.30231488,  0.5955519 ,  0.20989645, -0.28752038, -0.4138152 ,
        0.38020983,  0.00770567,  0.38003734, -0.17966715,  0.35888165,
       -0.25443712,  0.04367029, -0.02692631,  0.26533088,  0.22953166,
       -0.03343065, -0.0470276 ,  0.18654314, -0.00802147, -0.11168694,
       -0.09820867, -0.10806542,  0.32349876,  0.27227226, -0.04372728,
        0.09718672,  0.09889119,  0.143153  , -0.03100164,  0.1423716 ,
        0.17635132,  0.29027277,  0.01811146,  0.13421136,  0.00694755,
       -0.10776934,  0.68933666,  0.1327487 , -0.5016783 ,  0.63737005,
       -0.05147479, -0.15237482, -0.17462058, -0.36292893, -0.7200661 ,
        0.49903557, -0.59743625,  0.18768004,  0.22077252, -0.16

In [66]:
back_to_the_future_vector

array([ 6.16649259e-03, -1.55127214e-04, -1.63074327e-03,  2.50316015e-03,
        5.56584867e-03,  8.51607136e-03,  8.23458564e-03,  5.70871215e-03,
        7.79189263e-03,  4.50550113e-03,  1.15816556e-02, -1.56313588e-03,
        1.07621206e-02,  2.88502057e-03, -5.19872317e-03,  1.81392848e-03,
        3.66800069e-03, -2.47774791e-04,  2.46623554e-03,  1.99158257e-03,
        3.30098136e-03,  1.22684659e-02,  1.23280883e-02,  2.44318624e-04,
       -6.55325782e-03,  5.42585924e-03,  9.40117519e-03,  1.08957253e-02,
        1.03975646e-02,  7.83380773e-03,  7.50456611e-03, -2.33553772e-04,
        1.22797508e-02,  3.65401083e-03, -1.43076782e-03,  1.04038585e-02,
       -1.18723027e-02,  8.30481015e-03,  3.30412295e-03, -8.24906107e-04,
       -4.21340158e-03, -1.24835584e-03,  2.57598027e-03,  8.75945855e-03,
        2.13279016e-02,  8.55397061e-03,  1.74126169e-03,  2.90749507e-04,
        5.84053434e-03, -8.08418670e-04,  1.47161575e-03,  1.01422183e-02,
        2.79543805e-03,  

### 선호도 파악

In [67]:
np.dot(jang_vector, back_to_the_future_vector)

0.041036636

In [74]:
toy = title_to_idx['aliens (1986)']
toy_vector = als_model.item_factors[toy]
np.dot(jang_vector, toy_vector)

0.33845076

In [69]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [75]:
get_similar_title('aliens (1986)') 

['aliens (1986)',
 'terminator, the (1984)',
 'alien (1979)',
 'predator (1987)',
 'blade runner (1982)',
 'die hard (1988)',
 'abyss, the (1989)',
 'mad max 2 (a.k.a. the road warrior) (1981)',
 'terminator 2: judgment day (1991)',
 'full metal jacket (1987)']

In [77]:
get_similar_title('santa claus: the movie (1985)') 

['santa claus: the movie (1985)',
 'herbie goes bananas (1980)',
 'boy who could fly, the (1986)',
 'benji the hunted (1987)',
 'bogus (1996)',
 'neverending story ii: the next chapter, the (1990)',
 'apple dumpling gang rides again, the (1979)',
 'incredible journey, the (1963)',
 'cocoon: the return (1988)',
 'babes in toyland (1961)']

### 정말 신기하다. 에일리언은 공상과학 영화를 추천해주고 산타클로스는 소년영화같은것들을 추천해준다.

### 내가 가장 좋아할 만한 영화들을 추천받아보자.

In [79]:
user = user_to_idx['jang']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

terminator, the (1984) 0000.277
alien (1979) 0000.265
mad max 2 (a.k.a. the road warrior) (1981) 0000.240
blade runner (1982) 0000.202
predator (1987) 0000.194
terminator 2: judgment day (1991) 0000.188
mad max (1979) 0000.180
hamlet (1996) 0000.172
star wars: episode vi - return of the jedi (1983) 0000.170
star wars: episode v - the empire strikes back (1980) 0000.154
die hard (1988) 0000.151
abyss, the (1989) 0000.147
full metal jacket (1987) 0000.128
crimson tide (1995) 0000.127
alien³ (1992) 0000.119
robocop (1987) 0000.113
platoon (1986) 0000.113
glory (1989) 0000.111
big trouble in little china (1986) 0000.107
starman (1984) 0000.105


In [80]:
rihanna = title_to_idx['die hard (1988)']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('aliens (1986)', 0.15829495964424414),
 ('rollercoaster (1977)', 0.006924629526236707),
 ('santa claus: the movie (1985)', 0.006767510826457823),
 ('kull the conqueror (1997)', -0.005198854573900591),
 ('hamlet (1990)', -0.0167103194236004)]

### 참 재밌는 실습이었다. 하지만 내가 추천받을만한 영화중에 에일리언이 가장 인기있는 영화라 에일리언에 중점적으로 관계를 파악해 영화를 추천해줘서 카테고리별로 골고루 나오는게 아닌, 에일리언 중심으로 영화를 추천해주는게 안타까웠다. 이것으로 exploration15를 마친다.